In [0]:
#importing relevant libraries for the my analysis
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.preprocessing import StandardScaler
sns.set()

In [0]:
url='https://github.com/internbuddy/assignment/blob/master/ml-dl-ds/assignment_v2/data_science_sample_data_v2.xlsx?raw=true'
#Data upload directly from github repo

In [22]:
data = pd.read_excel(url)
data.head()
#loading the data into pandas dataframe and viewing it.

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Data Science (out of 3),Other skills,Institute,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
0,DS0001,Bangalore,1,0,3,"Machine Learning, Arduino, C Programming, CSS,...",Global Academy of Technology,Bachelor of Engineering (B.E),Electrical and Electronics Engineering,2019,NaN,7.73/10,NaN,NaN
1,DS0002,Mumbai,2,1,2,"AutoCAD, MS-Office, Machine Learning, Microsof...","Aegis School Of Business, Data Science, Cyber ...",NaN,PGP,2020,NaN,68.00/100,NaN,NaN
2,DS0003,Mumbai,2,0,0,"C++ Programming, Data Structures, Image Proces...","VJTI, Mumbai",Bachelor of Technology (B.Tech),Information Systems,2018,NaN,8.85/10,91.40/91.40,9.40/9.40
3,DS0004,Dhanbad,2,0,2,"Algorithms, C++ Programming, Data Structures, ...",IIT (ISM) Dhanbad,Integrated M.Tech,Mathematics and Computing,2021,NaN,8.40/10,91.80/91.80,10.00/10.00
4,DS0005,Bangalore,2,0,0,"MS-Word, Python, SQL, MS-Excel",Vvce,Bachelor of Engineering (B.E),Electronics and Communication,2018,NaN,NaN,NaN,NaN


In [23]:
data.describe(include='all')

,Application_ID,Current City,Python (out of 3),R Programming (out of 3),Data Science (out of 3),Other skills,Institute,Degree,Stream,Current Year Of Graduation,Performance_PG,Performance_UG,Performance_12,Performance_10
count,611,611,611.000000,611.000000,611.000000,601,611,575,580,611.000000,128,533,363,339
unique,611,70,NaN,NaN,NaN,513,439,42,120,NaN,83,269,204,155
top,DS0180,Bangalore,NaN,NaN,NaN,"Machine Learning, Python",Great Lakes Institute of Management,Bachelor of Technology (B.Tech),Computer Science,NaN,8.00/10,70.00/100,86.00/86.00,10.00/10.00
freq,1,334,NaN,NaN,NaN,5,21,168,72,NaN,6,15,8,24
mean,NaN,NaN,1.788871,0.582651,0.872340,NaN,NaN,NaN,NaN,2018.487725,NaN,NaN,NaN,NaN
std,NaN,NaN,0.780673,0.918459,1.014681,NaN,NaN,NaN,NaN,2.573082,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,2004.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,1.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,2018.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.000000,1.000000,2.000000,NaN,NaN,NaN,NaN,2020.000000,NaN,NaN,NaN,NaN


In [24]:
cleaned_data=data.drop(['Performance_PG',	'Performance_UG','Performance_12'	,'Performance_10','Current City','Stream','Institute'],axis=1)
cleaned_data.head()
#cleaning data and removing all the insignificant values out of model which are not included in our shortlisting criteria.

,Application_ID,Python (out of 3),R Programming (out of 3),Data Science (out of 3),Other skills,Degree,Current Year Of Graduation
0,DS0001,1,0,3,"Machine Learning, Arduino, C Programming, CSS,...",Bachelor of Engineering (B.E),2019
1,DS0002,2,1,2,"AutoCAD, MS-Office, Machine Learning, Microsof...",NaN,2020
2,DS0003,2,0,0,"C++ Programming, Data Structures, Image Proces...",Bachelor of Technology (B.Tech),2018
3,DS0004,2,0,2,"Algorithms, C++ Programming, Data Structures, ...",Integrated M.Tech,2021
4,DS0005,2,0,0,"MS-Word, Python, SQL, MS-Excel",Bachelor of Engineering (B.E),2018


In [0]:
cleaned_data=cleaned_data.rename(columns={"Python (out of 3)": "python", "R Programming (out of 3)": "R","Data Science (out of 3)": "data science"})
# renaming the data for ease

In [26]:
x=cleaned_data['Other skills'].str.split(",",expand=True)
f={'Machine Learning':3, 'Deep Learning':3 , 'NLP':3, 'Data Analytics':3 , 'AWS':3, 'SQL':3, 'MS-Excel':3, 'MySQL':3, 'NoSQL':3}
x1=x.replace(f,regex=True)
x1=x1.fillna(value=0)
x1=x1.apply(pd.to_numeric, errors='coerce')
x2=x1.sum(axis=1,numeric_only=True)
x2
#finding sum of skills










0      12.0
1       9.0
2       0.0
3      12.0
4       6.0
       ... 
606    15.0
607     6.0
608    12.0
609    18.0
610    12.0
Length: 611, dtype: float64

In [27]:
df=pd.concat([x2, cleaned_data], axis=1, sort=False)
df

,0,Application_ID,python,R,data science,Other skills,Degree,Current Year Of Graduation
0,12.0,DS0001,1,0,3,"Machine Learning, Arduino, C Programming, CSS,...",Bachelor of Engineering (B.E),2019
1,9.0,DS0002,2,1,2,"AutoCAD, MS-Office, Machine Learning, Microsof...",NaN,2020
2,0.0,DS0003,2,0,0,"C++ Programming, Data Structures, Image Proces...",Bachelor of Technology (B.Tech),2018
3,12.0,DS0004,2,0,2,"Algorithms, C++ Programming, Data Structures, ...",Integrated M.Tech,2021
4,6.0,DS0005,2,0,0,"MS-Word, Python, SQL, MS-Excel",Bachelor of Engineering (B.E),2018
...,...,...,...,...,...,...,...,...
606,15.0,DS0607,2,2,2,"AutoCAD, MS-Excel, Deep Learning, Machine Lear...",Bachelor of Engineering (B.E),2020
607,6.0,DS0608,1,0,0,"MS-Excel, HTML, Python, SQL",PG Diploma in Data Science,2020
608,12.0,DS0609,2,2,0,"Data Analytics, Machine Learning, Natural Lang...",Bachelor of Technology (B.Tech),2016
609,18.0,DS0610,3,2,2,"Data Analytics, Machine Learning, Python, C Pr...",Bachelor of Science (B.Sc),2018


In [28]:
df=df.drop('Other skills',axis=1)
df

,0,Application_ID,python,R,data science,Degree,Current Year Of Graduation
0,12.0,DS0001,1,0,3,Bachelor of Engineering (B.E),2019
1,9.0,DS0002,2,1,2,NaN,2020
2,0.0,DS0003,2,0,0,Bachelor of Technology (B.Tech),2018
3,12.0,DS0004,2,0,2,Integrated M.Tech,2021
4,6.0,DS0005,2,0,0,Bachelor of Engineering (B.E),2018
...,...,...,...,...,...,...,...
606,15.0,DS0607,2,2,2,Bachelor of Engineering (B.E),2020
607,6.0,DS0608,1,0,0,PG Diploma in Data Science,2020
608,12.0,DS0609,2,2,0,Bachelor of Technology (B.Tech),2016
609,18.0,DS0610,3,2,2,Bachelor of Science (B.Sc),2018


In [29]:
df=df.rename(columns={0:"sum of skills"})
df

,sum of skills,Application_ID,python,R,data science,Degree,Current Year Of Graduation
0,12.0,DS0001,1,0,3,Bachelor of Engineering (B.E),2019
1,9.0,DS0002,2,1,2,NaN,2020
2,0.0,DS0003,2,0,0,Bachelor of Technology (B.Tech),2018
3,12.0,DS0004,2,0,2,Integrated M.Tech,2021
4,6.0,DS0005,2,0,0,Bachelor of Engineering (B.E),2018
...,...,...,...,...,...,...,...
606,15.0,DS0607,2,2,2,Bachelor of Engineering (B.E),2020
607,6.0,DS0608,1,0,0,PG Diploma in Data Science,2020
608,12.0,DS0609,2,2,0,Bachelor of Technology (B.Tech),2016
609,18.0,DS0610,3,2,2,Bachelor of Science (B.Sc),2018


In [30]:
df['python']=df['python'].map({3:10, 2:7,1:3})
df['R']=df['R'].map({3:10, 2:7, 1:3})
df['data science']=df['data science'].map({3:10, 2:7, 1:3})
p=df['Degree'].unique()
p

array(['Bachelor of Engineering (B.E)', nan,
       'Bachelor of Technology (B.Tech)', 'Integrated M.Tech',
       'B.Tech (Hons.)', 'Master of Business Administration_(MBA)', 'MBA',
       'Master of Science (M.Sc)', 'Post Graduate Programme (PGP)',
       'Master of Computer Applications (MCA)',
       'Bachelor of Pharmacy (B.Pharm)', 'Integrated MCA',
       'Bachelor of Commerce (B.Com)', 'Master of Technology (M.Tech)',
       'Integrated B.Sc. & M.Sc.', 'Post Graduate Diploma',
       'Integrated M.Sc.', 'Master of Arts (M.A.)',
       'PG Diploma in Data Science', 'Bachelor of Science (B.Sc)',
       'Integrated B.Tech', 'Master of Statistics (M.Stat)',
       'Post Graduate Diploma in Management (P.G.D.M.)',
       'Master of Engineering (M.E)', 'Master Of Analytics',
       'Bachelor of Computer Applications (BCA)',
       'Integrated B.S. & M.S.', 'Bachelor of Science (B.Sc) (Hons.)',
       'Bachelor of Computer Science (B.C.S.)',
       'Master of Science (M.S.)', 'Master 

In [31]:
df['Degree']=df['Degree'].map({'Bachelor of Engineering (B.E)':'Ug','Bachelor of Technology (B.Tech)':'Ug','Integrated B.Tech & M.Tech':'Ug','Bachelor of Engineering (B.E) (Hons.)':'Ug','Master of Technology (M.Tech)':'pg','Master of Engineering (M.E)':'pg','Integrated M.Sc.':'pg','Master of Science (M.Sc) (Hons.)':'pg','Post Graduate Diploma in Big Data Analytics (PG-DBDA)':'pg' })

df



,sum of skills,Application_ID,python,R,data science,Degree,Current Year Of Graduation
0,12.0,DS0001,3.0,NaN,10.0,Ug,2019
1,9.0,DS0002,7.0,3.0,7.0,NaN,2020
2,0.0,DS0003,7.0,NaN,NaN,Ug,2018
3,12.0,DS0004,7.0,NaN,7.0,NaN,2021
4,6.0,DS0005,7.0,NaN,NaN,Ug,2018
...,...,...,...,...,...,...,...
606,15.0,DS0607,7.0,7.0,7.0,Ug,2020
607,6.0,DS0608,3.0,NaN,NaN,NaN,2020
608,12.0,DS0609,7.0,7.0,NaN,Ug,2016
609,18.0,DS0610,10.0,7.0,7.0,NaN,2018


In [32]:

df['new degree']=df['Current Year Of Graduation'].astype(str)+'_'+df['Degree']
df

,sum of skills,Application_ID,python,R,data science,Degree,Current Year Of Graduation,new degree
0,12.0,DS0001,3.0,NaN,10.0,Ug,2019,2019_Ug
1,9.0,DS0002,7.0,3.0,7.0,NaN,2020,NaN
2,0.0,DS0003,7.0,NaN,NaN,Ug,2018,2018_Ug
3,12.0,DS0004,7.0,NaN,7.0,NaN,2021,NaN
4,6.0,DS0005,7.0,NaN,NaN,Ug,2018,2018_Ug
...,...,...,...,...,...,...,...,...
606,15.0,DS0607,7.0,7.0,7.0,Ug,2020,2020_Ug
607,6.0,DS0608,3.0,NaN,NaN,NaN,2020,NaN
608,12.0,DS0609,7.0,7.0,NaN,Ug,2016,2016_Ug
609,18.0,DS0610,10.0,7.0,7.0,NaN,2018,NaN


In [33]:
df=df.drop(['Current Year Of Graduation','Degree'],axis=1)
df

,sum of skills,Application_ID,python,R,data science,new degree
0,12.0,DS0001,3.0,NaN,10.0,2019_Ug
1,9.0,DS0002,7.0,3.0,7.0,NaN
2,0.0,DS0003,7.0,NaN,NaN,2018_Ug
3,12.0,DS0004,7.0,NaN,7.0,NaN
4,6.0,DS0005,7.0,NaN,NaN,2018_Ug
...,...,...,...,...,...,...
606,15.0,DS0607,7.0,7.0,7.0,2020_Ug
607,6.0,DS0608,3.0,NaN,NaN,NaN
608,12.0,DS0609,7.0,7.0,NaN,2016_Ug
609,18.0,DS0610,10.0,7.0,7.0,NaN


In [34]:
new=df['new degree'].unique()
new

array(['2019_Ug', nan, '2018_Ug', '2021_Ug', '2022_Ug', '2020_Ug',
       '2017_Ug', '2020_pg', '2016_pg', '2014_Ug', '2016_Ug', '2022_pg',
       '2015_pg', '2019_pg', '2015_Ug', '2013_Ug', '2017_pg', '2018_pg',
       '2021_pg', '2006_Ug', '2009_Ug', '2004_Ug', '2023_pg', '2012_pg',
       '2008_pg', '2012_Ug'], dtype=object)

In [35]:
df['new degree']=df['new degree'].map({'2020_Ug':10,'2019_Ug':8,'2018_Ug':5,'2017_Ug':5,'2016_Ug':5,'2015_Ug':5,'2014_Ug':5,'2013_Ug':5,'2012_Ug':5,'2009_Ug':5,'2006_Ug':5,'2004_Ug':5,'2020_pg':7,'2019_pg':3,'2018_pg':3,'2017_pg':3,'2016_pg':3,'2015_pg':3,'2012_pg':3,'2008_pg':3})
df

,sum of skills,Application_ID,python,R,data science,new degree
0,12.0,DS0001,3.0,NaN,10.0,8.0
1,9.0,DS0002,7.0,3.0,7.0,NaN
2,0.0,DS0003,7.0,NaN,NaN,5.0
3,12.0,DS0004,7.0,NaN,7.0,NaN
4,6.0,DS0005,7.0,NaN,NaN,5.0
...,...,...,...,...,...,...
606,15.0,DS0607,7.0,7.0,7.0,10.0
607,6.0,DS0608,3.0,NaN,NaN,NaN
608,12.0,DS0609,7.0,7.0,NaN,5.0
609,18.0,DS0610,10.0,7.0,7.0,NaN


In [36]:
df['final data science score']=df.sum(axis=1,numeric_only=True)
df

,sum of skills,Application_ID,python,R,data science,new degree,final data science score
0,12.0,DS0001,3.0,NaN,10.0,8.0,33.0
1,9.0,DS0002,7.0,3.0,7.0,NaN,26.0
2,0.0,DS0003,7.0,NaN,NaN,5.0,12.0
3,12.0,DS0004,7.0,NaN,7.0,NaN,26.0
4,6.0,DS0005,7.0,NaN,NaN,5.0,18.0
...,...,...,...,...,...,...,...
606,15.0,DS0607,7.0,7.0,7.0,10.0,46.0
607,6.0,DS0608,3.0,NaN,NaN,NaN,9.0
608,12.0,DS0609,7.0,7.0,NaN,5.0,31.0
609,18.0,DS0610,10.0,7.0,7.0,NaN,42.0


In [38]:
df['data science application considered or not'] = np.where(df['final data science score']>=40, 'yes', 'no')
df

,sum of skills,Application_ID,python,R,data science,new degree,final data science score,data science application considered or not
0,12.0,DS0001,3.0,NaN,10.0,8.0,33.0,no
1,9.0,DS0002,7.0,3.0,7.0,NaN,26.0,no
2,0.0,DS0003,7.0,NaN,NaN,5.0,12.0,no
3,12.0,DS0004,7.0,NaN,7.0,NaN,26.0,no
4,6.0,DS0005,7.0,NaN,NaN,5.0,18.0,no
...,...,...,...,...,...,...,...,...
606,15.0,DS0607,7.0,7.0,7.0,10.0,46.0,yes
607,6.0,DS0608,3.0,NaN,NaN,NaN,9.0,no
608,12.0,DS0609,7.0,7.0,NaN,5.0,31.0,no
609,18.0,DS0610,10.0,7.0,7.0,NaN,42.0,yes
